# BiLstm手写识别[<img src="http://ogtxggxo6.bkt.clouddn.com/home.jpg?imageView2/2/h/30" align="right"/>](http://petersansan.top)
```
Author: PeterSansan
Times :2017-04-23
```

In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./../mnist/MNIST_data/", one_hot=True)

Extracting ./../mnist/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./../mnist/MNIST_data/t10k-labels-idx1-ubyte.gz


这里的处理与LSTM程序中的一样，也是把一个图像分成28个序列，每个序列长28
```
To classify images using a bidirectional reccurent neural network, we consider
every image row as a sequence of pixels. Because MNIST image shape is 28*28px,
we will then handle 28 sequences of 28 steps for every sample.
```

In [2]:
# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 28 # MNIST data input (img shape: 28*28)
n_steps = 28 # timesteps
n_hidden = 128 # hidden layer num of features
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_input])
y = tf.placeholder("float", [None, n_classes])

# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of foward + backward cells
    'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print("done")

done


In [3]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `bidirectional_rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_input)
    
    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = BiRNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [4]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_input))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc)
        step += 1
    print "Optimization Finished!"

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
    test_label = mnist.test.labels[:test_len]
    print "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label})

Iter 1280, Minibatch Loss= 1.941263, Training Accuracy= 0.35156
Iter 2560, Minibatch Loss= 1.426232, Training Accuracy= 0.53906
Iter 3840, Minibatch Loss= 1.282442, Training Accuracy= 0.57812
Iter 5120, Minibatch Loss= 1.151229, Training Accuracy= 0.66406
Iter 6400, Minibatch Loss= 0.758029, Training Accuracy= 0.76562
Iter 7680, Minibatch Loss= 0.828590, Training Accuracy= 0.71875
Iter 8960, Minibatch Loss= 0.550428, Training Accuracy= 0.82031
Iter 10240, Minibatch Loss= 0.512208, Training Accuracy= 0.82812
Iter 11520, Minibatch Loss= 0.449139, Training Accuracy= 0.88281
Iter 12800, Minibatch Loss= 0.369014, Training Accuracy= 0.85938
Iter 14080, Minibatch Loss= 0.447119, Training Accuracy= 0.82812
Iter 15360, Minibatch Loss= 0.515295, Training Accuracy= 0.83594
Iter 16640, Minibatch Loss= 0.354486, Training Accuracy= 0.89062
Iter 17920, Minibatch Loss= 0.475124, Training Accuracy= 0.83594
Iter 19200, Minibatch Loss= 0.356262, Training Accuracy= 0.86719
Iter 20480, Minibatch Loss= 0.24

双向LSTM的代码几乎与单向LSTM代码一样，只是模型定向时使用的函数不同，但是这里封装了LSTM的实现，并不能更为详细的看到了内部的实现，只能通过查看源码或相关论文。

[<img src="http://ogtxggxo6.bkt.clouddn.com/home.jpg?imageView2/2/h/30" align="left"/>](http://petersansan.top)